In [4]:
# initial install
%pip install pandas
%pip install openpyxl

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
# install packages
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
# update packages
%pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [7]:
# import packages + load dataframe
import pandas as pd

# Load CSV file into DataFrame
df = pd.read_csv('data.csv')


C:\Users\bryan\AppData\Local\Temp\ipykernel_15988\4076321326.py:5: DtypeWarning: Columns (690,691,692,703,706,708,709,710,711,712,713,714,715,716,717,718,719,720,721,1172,1174,1185,1214) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data.csv')


# Data Exploration

In [8]:
# analyze dataframe

# print(list(df.columns))

In [9]:
# View ranges of data in each column

"""
for col in df.columns:
    unique_values = df[col].dropna().unique()  # Drop NaNs if you don’t want them in the list
    unique_values_list = [float(val) if isinstance(val, (float, int)) else val for val in unique_values]
    print(f"{col}: {unique_values_list}")
"""


'\nfor col in df.columns:\n    unique_values = df[col].dropna().unique()  # Drop NaNs if you don’t want them in the list\n    unique_values_list = [float(val) if isinstance(val, (float, int)) else val for val in unique_values]\n    print(f"{col}: {unique_values_list}")\n'

# Clean Dataframe

In [10]:
# 1. Drop rows that aren't questions, or questions that have text values.
columns_to_keep = [col for col in df.columns if ('Q' in col or 'QN' in col) and 'TEXT' not in col]
columns_to_drop = [col for col in df.columns if col not in columns_to_keep]
df = df[columns_to_keep]

# Print dropped columns
# print("Dropped columns:", columns_to_drop)

In [11]:
# 2. Drop all QN columns

columns_to_drop = [col for col in df.columns if col.startswith("QN")]
columns_to_keep = [col for col in df.columns if col.startswith("Q") and not col.startswith("QN")]
df_old = df[columns_to_drop]
df = df[columns_to_keep]

# print("Dropped columns:", columns_to_drop)
# print("\n")
# print("Kept columns:", columns_to_keep)


In [12]:
# 3. Merge all multi-choice questions into one column.

In [13]:
# 3: Question 4a

# Debug print to display the value counts of Q4a before transformations
print("Initial value counts in Q4a:")
print(df['Q4a'].value_counts(dropna=False))
print("Initial value counts in Q4b:")
print(df['Q4b'].value_counts(dropna=False))

# Transformation code
main_column = 'Q4a'
additional_columns = ['Q4b', 'Q4c', 'Q4d', 'Q4e']

for index, row in df.iterrows():
    for col in additional_columns:
        try:
            if float(row[col]) == 1.0:
                df.at[index, main_column] = col[-1]  # Use suffix (b, c, d, or e)
                break
        except (ValueError, TypeError):
            continue

df = df.drop(columns=additional_columns)

# Display the updated value counts in Q4a after transformations
print("Updated value counts in Q4a:")
print(df['Q4a'].value_counts(dropna=False))

Initial value counts in Q4a:
Q4a
1      15287
NaN     6266
N        492
Z         24
Name: count, dtype: int64
Initial value counts in Q4b:
Q4b
NaN    17689
1       3864
N        492
Z         24
Name: count, dtype: int64
Updated value counts in Q4a:
Q4a
1    15287
b     3864
e     1863
N      492
c      418
d      121
Z       24
Name: count, dtype: int64


Display the initial values for all the columns that will be making up the new column for Q5

In [ ]:
additional_columns = ['Q5a', 'Q5b', 'Q5c', 'Q5d', 'Q5e']

for col in additional_columns:
    print(f"Value counts for column {col}:")
    print(df[col].value_counts(dropna=False))
    print("-" * 50)



Value counts for column Q5a:
Q5a
NaN    17990
1       2438
N       1588
Z         53
Name: count, dtype: int64
--------------------------------------------------
Value counts for column Q5b:
Q5b
NaN    17535
1       2893
N       1588
Z         53
Name: count, dtype: int64
--------------------------------------------------
Value counts for column Q5c:
Q5c
NaN    16187
1       4241
N       1588
Z         53
Name: count, dtype: int64
--------------------------------------------------
Value counts for column Q5d:
Q5d
NaN    19736
N       1588
1        692
Z         53
Name: count, dtype: int64
--------------------------------------------------
Value counts for column Q5e:
Q5e
1      12433
NaN     7995
N       1588
Z         53
Name: count, dtype: int64
--------------------------------------------------


In [15]:
# 3: Question 5

# Debug print to display the value counts of QN5A before transformations
print("Initial value counts in Q5a:")
print(df['Q5a'].value_counts(dropna=False))

# Define the main column and additional columns to check for Q5
main_column = 'Q5'
additional_columns = ['Q5a', 'Q5b', 'Q5c', 'Q5d', 'Q5e']

# Initialize Q5 column with values from Q5a where appropriate
df[main_column] = df['Q5a']

# Transformation logic for Q5
for index, row in df.iterrows():
    value_assigned = False
    for col in additional_columns:
        try:
            # Check if the current column has a 1.0 value
            if float(row[col]) == 1.0:
                df.at[index, main_column] = col[-1]  # Set Q5 to the suffix (a, b, c, d, or e)
                value_assigned = True
                break
        except (ValueError, TypeError):
            continue

    # If no 1.0 was found, retain 'Z' or 'N' based on values in Q5a to Q5e
    if not value_assigned:
        if any(row[col] == 'N' for col in additional_columns):
            df.at[index, main_column] = 'N'
        elif any(row[col] == 'Z' for col in additional_columns):
            df.at[index, main_column] = 'Z'

# Drop the additional columns
df = df.drop(columns=additional_columns)

# Display the updated value counts in Q5 after transformations
print("Updated value counts in Q5:")
print(df['Q5'].value_counts(dropna=False))


Initial value counts in Q5a:
Q5a
NaN    17990
1       2438
N       1588
Z         53
Name: count, dtype: int64
Updated value counts in Q5:
Q5
e    10949
c     3854
b     2750
a     2438
N     1588
d      437
Z       53
Name: count, dtype: int64


*****This logic above isn't counting all the actual instances into the new column

Display the initial values for all the columns that will be making up the new column for Q11

In [16]:
additional_columns = ['Q11a', 'Q11b', 'Q11c', 'Q11d', 'Q11e', 'Q11f', 'Q11g', 'Q11h', 'Q11i', 'Q11j', 'Q11k', 'Q11l', 'Q11m', 'Q11n']

for col in additional_columns:
    print(f"Value counts for column {col}:")
    print(df[col].value_counts(dropna=False))
    print("-" * 50)



Value counts for column Q11a:
Q11a
S      18518
1       1654
NaN     1631
Z        213
N         42
1.0       11
Name: count, dtype: int64
--------------------------------------------------
Value counts for column Q11b:
Q11b
S      18518
NaN     2512
1        780
Z        213
N         42
1.0        4
Name: count, dtype: int64
--------------------------------------------------
Value counts for column Q11c:
Q11c
S      18518
NaN     3190
Z        213
1        102
N         42
1.0        4
Name: count, dtype: int64
--------------------------------------------------
Value counts for column Q11d:
Q11d
S      18518
NaN     3202
Z        213
1         91
N         42
1.0        3
Name: count, dtype: int64
--------------------------------------------------
Value counts for column Q11e:
Q11e
S      18518
NaN     3116
Z        213
1        176
N         42
1.0        4
Name: count, dtype: int64
--------------------------------------------------
Value counts for column Q11f:
Q11f
S      18518
Na

In [17]:
# 3: Question 11

# Debug print to display the initial value counts in Q11a before transformation
print("Initial value counts in Q11a:")
print(df['Q11a'].value_counts(dropna=False))

# Define the main column and all Q11* columns to check for Q11
main_column = 'Q11'
additional_columns = ['Q11a', 'Q11b', 'Q11c', 'Q11d', 'Q11e', 'Q11f', 'Q11g', 'Q11h', 'Q11i', 'Q11j', 'Q11k', 'Q11l', 'Q11m', 'Q11n']

# Initialize Q11 column with values from Q11a where appropriate
df[main_column] = df['Q11a']

# Transformation logic for Q11
for index, row in df.iterrows():
    value_assigned = False
    for col in additional_columns:
        try:
            # Check if the current column has a 1.0 value
            if float(row[col]) == 1.0:
                df.at[index, main_column] = col[-1].lower()  # Set Q11 to the suffix in lowercase (a, b, c, etc.)
                value_assigned = True
                break
        except (ValueError, TypeError):
            continue

    # If no 1.0 was found, retain 'Z' or 'N' based on values in Q11a to Q11n
    if not value_assigned:
        if any(row[col] == 'N' for col in additional_columns):
            df.at[index, main_column] = 'N'
        elif any(row[col] == 'Z' for col in additional_columns):
            df.at[index, main_column] = 'Z'

# Drop all Q11* columns, leaving only Q11
df = df.drop(columns=additional_columns)

# Display the updated value counts in Q11 after transformations
print("Updated value counts in Q11:")
print(df['Q11'].value_counts(dropna=False))


Initial value counts in Q11a:
Q11a
S      18518
1       1654
NaN     1631
Z        213
N         42
1.0       11
Name: count, dtype: int64
Updated value counts in Q11:
Q11
S    18518
a     1665
k      479
b      416
Z      213
l      191
n      141
m       73
j       73
h       70
N       42
i       36
c       34
f       33
g       33
e       32
d       20
Name: count, dtype: int64


***** This is also not counting all the actual instances for each option to the new column. Ex: original column Q11m had 651 instances, but in the new column it says there were only 73 

Display the initial values for all the columns that will be making up the new column for Q12

In [18]:
additional_columns = ['Q12a', 'Q12b', 'Q12c', 'Q12d', 'Q12e', 'Q12f', 'Q12g', 'Q12h', 'Q12i', 'Q12j', 'Q12k', 'Q12l', 'Q12m', 'Q12n']


for col in additional_columns:
    print(f"Value counts for column {col}:")
    print(df[col].value_counts(dropna=False))
    print("-" * 50)



Value counts for column Q12a:
Q12a
S      20301
NaN     1043
1        430
Z        219
N         76
Name: count, dtype: int64
--------------------------------------------------
Value counts for column Q12b:
Q12b
S      20301
NaN     1294
Z        219
1        179
N         76
Name: count, dtype: int64
--------------------------------------------------
Value counts for column Q12c:
Q12c
S      20301
NaN     1394
Z        219
1         79
N         76
Name: count, dtype: int64
--------------------------------------------------
Value counts for column Q12d:
Q12d
S      20301
NaN     1383
Z        219
1         90
N         76
Name: count, dtype: int64
--------------------------------------------------
Value counts for column Q12e:
Q12e
S      20301
NaN     1353
Z        219
1        120
N         76
Name: count, dtype: int64
--------------------------------------------------
Value counts for column Q12f:
Q12f
S      20301
NaN     1413
Z        219
N         76
1         60
Name: count, dt

In [19]:
# Question 12

# Debug print to display the initial value counts in Q12a before transformation
print("Initial value counts in Q12a:")
print(df['Q12a'].value_counts(dropna=False))

# Define the main column and all Q12* columns to check for Q12
main_column = 'Q12'
additional_columns = ['Q12a', 'Q12b', 'Q12c', 'Q12d', 'Q12e', 'Q12f', 'Q12g', 'Q12h', 'Q12i', 'Q12j', 'Q12k', 'Q12l', 'Q12m', 'Q12n']

# Initialize Q12 with values from Q12a where appropriate
df[main_column] = df['Q12a']

# Transformation logic for Q12
for index, row in df.iterrows():
    value_assigned = False
    for col in additional_columns:
        try:
            # Check if the current column has a 1.0 value
            if float(row[col]) == 1.0:
                df.at[index, main_column] = col[-1].lower()  # Set Q12 to the suffix in lowercase (a, b, c, etc.)
                value_assigned = True
                break
        except (ValueError, TypeError):
            continue

    # If no 1.0 was found, retain 'Z' or 'N' based on values in Q12a to Q12n
    if not value_assigned:
        if any(row[col] == 'N' for col in additional_columns):
            df.at[index, main_column] = 'N'
        elif any(row[col] == 'Z' for col in additional_columns):
            df.at[index, main_column] = 'Z'

# Drop all Q12* columns, leaving only Q12
df = df.drop(columns=additional_columns)

# Display the updated value counts in Q12 after transformations
print("Updated value counts in Q12:")
print(df['Q12'].value_counts(dropna=False))

Initial value counts in Q12a:
Q12a
S      20301
NaN     1043
1        430
Z        219
N         76
Name: count, dtype: int64
Updated value counts in Q12:
Q12
S    20301
a      430
l      291
Z      219
n      149
m      129
j       76
N       76
k       68
h       67
b       64
g       47
c       40
i       39
e       36
d       28
f        9
Name: count, dtype: int64


*** This is also not counting all of instances and putting it into the new column. Ex: Q12b had 179 instances but in the merged column it states only 64 instances for b 

Now to merge the ones for question 14 
first display the initial values for all the columns that will get merged

In [20]:
columns_to_merge = ['Q14a', 'Q14b', 'Q14c', 'Q14d', 'Q14e', 'Q14f', 'Q14g', 'Q14h', 
                    'Q14i', 'Q14j', 'Q14k', 'Q14l', 'Q14m', 'Q14n', 'Q14o', 'Q14p']

for col in columns_to_merge:
    print(f"Value counts for column {col}:")
    print(df[col].value_counts(dropna=False))
    print("-" * 50)


Value counts for column Q14a:
Q14a
S      20301
NaN     1396
Z        231
1        115
N         26
Name: count, dtype: int64
--------------------------------------------------
Value counts for column Q14b:
Q14b
S      20301
NaN     1272
1        239
Z        231
N         26
Name: count, dtype: int64
--------------------------------------------------
Value counts for column Q14c:
Q14c
S      20301
1        839
NaN      672
Z        231
N         26
Name: count, dtype: int64
--------------------------------------------------
Value counts for column Q14d:
Q14d
S      20301
NaN     1209
1        302
Z        231
N         26
Name: count, dtype: int64
--------------------------------------------------
Value counts for column Q14e:
Q14e
S      20301
NaN     1353
Z        231
1        158
N         26
Name: count, dtype: int64
--------------------------------------------------
Value counts for column Q14f:
Q14f
S      20301
NaN     1274
1        237
Z        231
N         26
Name: count, dt

In [21]:
# Step 1: Initialize the columns to merge
columns_to_merge = ['Q14a', 'Q14b', 'Q14c', 'Q14d', 'Q14e', 'Q14f', 
                    'Q14g', 'Q14h', 'Q14i', 'Q14j', 'Q14k', 'Q14l', 
                    'Q14m', 'Q14n', 'Q14o', 'Q14p']



# Step 2: Create a new column 'Q14' to store selected options
df['Q14'] = None

# Step 3: Populate the new column by counting each option independently
for index, row in df.iterrows():
    # Initialize a list to collect selected options
    selected_options = []
    
    # Check each column for a value of 1.0 and add the corresponding option to the list
    for col in columns_to_merge:
        try:
            if float(row[col]) == 1.0:
                selected_options.append(col[-1].lower())
        except (ValueError, TypeError):
            continue
    
    # If multiple options were selected, count each one independently
    if selected_options:
        for option in selected_options:
            # Increment the count for each option independently
            if pd.isnull(df.at[index, 'Q14']):
                df.at[index, 'Q14'] = option
            else:
                df.at[index, 'Q14'] += ',' + option
    else:
        # Handle 'Z' or 'N' cases if no options were selected
        if any(row[col] == 'N' for col in columns_to_merge):
            df.at[index, 'Q14'] = 'N'
        elif any(row[col] == 'Z' for col in columns_to_merge):
            df.at[index, 'Q14'] = 'Z'

# Step 4: Split the comma-separated values into individual rows for counting
df['Q14'] = df['Q14'].str.split(',')
df = df.explode('Q14')

# Step 5: Display the value counts for the new 'Q14' column
print("\nUpdated value counts in Q14:")
print(df['Q14'].value_counts(dropna=False))



Updated value counts in Q14:
Q14
None    20301
c         839
p         367
n         333
d         302
o         275
b         239
f         237
Z         231
j         176
e         158
h         155
l         136
a         115
k         103
g          85
m          78
i          69
N          26
Name: count, dtype: int64


Now right here all the values for the merged column match the original values that made it up ***